In [1]:
!git clone https://github.com/starringkarthik/3D-Reconstruction.git

Cloning into '3D-Reconstruction'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 170 (delta 27), reused 31 (delta 12), pack-reused 113
Receiving objects: 100% (170/170), 90.41 MiB | 20.73 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Updating files: 100% (80/80), done.


In [2]:
import os
os.chdir("/content/3D-Reconstruction/pifuhd/")
!sh ./scripts/download_trained_model.sh

+ mkdir -p checkpoints
+ cd checkpoints
+ wget https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt pifuhd.pt
--2024-04-26 12:19:22--  https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.33, 13.227.219.70, 13.227.219.59, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548375177 (1.4G) [application/octet-stream]
Saving to: ‘pifuhd.pt’

pifuhd.pt           100%[===================>]   1.44G  94.6MB/s    in 13s     

2024-04-26 12:19:35 (118 MB/s) - ‘pifuhd.pt’ saved [1548375177/1548375177]

--2024-04-26 12:19:35--  http://pifuhd.pt/
Resolving pifuhd.pt (pifuhd.pt)... failed: Name or service not known.
wget: unable to resolve host address ‘pifuhd.pt’
FINISHED --2024-04-26 12:19:35--
Total wall clock time: 13s
Downloaded: 1 files, 1.4G in 13s (118 MB/s)


In [3]:
import os
os.chdir("/content/3D-Reconstruction")
!pip install -q streamlit
!pip install obj2html
!chmod +x cloudflared-linux-amd64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for obj2html: filename=obj2html-0.14-py3-none-any.whl size=5095 sha256=0372d8df279f2a0e5595394f3a8be4e3320774e6479ee210e52e285d4601c0d5
  Stored in directory: /root/.cache/pip/wheels/b3/39/0f/aa3787183fb84c7c6d71ef487e26d6101c5616e025c4c05f4b
Successfully built obj2html


In [4]:
!nohup /content/3D-Reconstruction/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"
!streamlit run /content/3D-Reconstruction/app.py &>/content/logs.txt &

nohup: appending output to 'nohup.out'


In [5]:
directory_path = '/content/3D-Reconstruction/pifuhd/sample_images/'
filename = None
updated_time = 0
for entry in os.scandir(directory_path):
  if entry.is_file():
    mod_time = entry.stat().st_mtime_ns
    if mod_time > updated_time:
      filename = entry.name
#try:
image_path = '/content/3D-Reconstruction/pifuhd/sample_images/%s' % filename
#except:
#  image_path = '/content/pifuhd/sample_images/test.png'
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

obj_path = '/content/3D-Reconstruction/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = '/content/3D-Reconstruction/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = '/content/3D-Reconstruction/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = '/content/3D-Reconstruction/pifuhd/results/pifuhd_final/result_%s_256_display.mp4' % file_name
print(filename)
os.chdir("/content/3D-Reconstruction/lightweight-human-pose-estimation.pytorch/")

import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)

            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:

              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

get_rect(net.cuda(), [image_path], 512)
os.chdir("/content/3D-Reconstruction/pifuhd/")
!python -m apps.simple_test -r 256 --use_rect -i $image_dir
os.chdir("/content/3D-Reconstruction")


human.jpg
Resuming from  ./checkpoints/pifuhd.pt
test data size:  1
initialize network with normal
initialize network with normal
generate mesh (test) ...
  0% 0/1 [00:00<?, ?it/s]./results/pifuhd_final/recon/result_human_256.obj
100% 1/1 [00:07<00:00,  7.44s/it]


In [6]:
%%writefile app.py
import os
from obj2html import obj2html
from IPython.display import display, HTML
import streamlit as st

os.chdir('/content/3D-Reconstruction/pifuhd/results/pifuhd_final/recon')

st.title("3D Model")

obj_files = [f for f in os.listdir() if f.endswith(".obj")]
obj_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

if obj_files:
    most_recent_obj = obj_files[0]
    obj2html(most_recent_obj, 'index.html')

    st.write("3D Reconstructed Model:")
    st.components.v1.html(open('index.html').read(), width=320, height=200)

    st.markdown("Download Object File:")
    with open(most_recent_obj, 'rb') as f:
        st.download_button('Download model.obj', f, key=most_recent_obj, file_name=most_recent_obj)

    st.link_button("Visualize using Online 3D Viewer", "https://3dviewer.net/")

else:
    st.write("No .obj files found in the directory.")

Overwriting app.py


# backup

In [ ]:
!npm install localtunnel

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!streamlit run /content/3D-Reconstruction/app.py &>/content/logs.txt &
!npx localtunnel --port 8501